# Hot Delivery - Part I
Team 3

## Importing Packages

In [ ]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pulp
from scipy import stats
import datetime

# Initialize seaborn (for plotting)
sns.set()

## Loading and Inspecting Data


In [ ]:
#Case 1: orders A

dfOrdersA= pd.read_csv("part1_ordersA.csv",index_col=0)
dfOrdersA

,customer
restaurant,
Downtown Toronto (Underground city),Downtown Toronto (Central Bay Street)


In [ ]:
#Case 2: orders B

dfOrdersB= pd.read_csv("part1_ordersB.csv",index_col=0)
dfOrdersB

,customer
restaurant,
Downtown Toronto (Central Bay Street),Downtown Toronto (Underground city)
Etobicoke Northwest (Clairville / Humberwood / Woodbine Downs / West Humber / Kipling Heights / Rexdale / Elms / Tandridge / Old Rexdale),Etobicoke (South Steeles / Silverstone / Humbe...
York (Cedarvale),Central Toronto (The Annex / North Midtown / Y...
Downtown Toronto (Central Bay Street),Downtown Toronto (Richmond / Adelaide / King)
Downtown Toronto (Richmond / Adelaide / King),Downtown Toronto (St. James Town / Cabbagetown)


In [ ]:
# Average Distance between neighborhoods

dfDistance = pd.read_csv("distances.csv")
dfDistance

,origin,destination,distance
0,Scarborough (Malvern / Rouge River),Scarborough (Rouge Hill / Port Union / Highlan...,3.931478
1,Scarborough (Malvern / Rouge River),Scarborough (Guildwood / Morningside / Ellesmere),4.864191
2,Scarborough (Malvern / Rouge River),Scarborough (Woburn),4.778347
3,Scarborough (Malvern / Rouge River),Scarborough (Cedarbrae),6.009861
4,Scarborough (Malvern / Rouge River),Scarborough (Eglinton),7.876162
...,...,...,...
10297,Etobicoke (South Steeles / Silverstone / Humbe...,Etobicoke (Westmount),6.971037
10298,Etobicoke Northwest (Clairville / Humberwood /...,Etobicoke (Westmount),5.185601
10299,Etobicoke (South Steeles / Silverstone / Humbe...,Etobicoke (Kingsview Village / St. Phillips / ...,6.390667
10300,Etobicoke Northwest (Clairville / Humberwood /...,Etobicoke (Kingsview Village / St. Phillips / ...,3.793723


In [ ]:
#regional data

dfRegions = pd.read_csv("regions.csv")
dfRegions

,name,province,code,latitude,longitude
0,Scarborough (Malvern / Rouge River),Ontario,M1B,43.8113,-79.1930
1,Scarborough (Rouge Hill / Port Union / Highlan...,Ontario,M1C,43.7878,-79.1564
2,Scarborough (Guildwood / Morningside / Ellesmere),Ontario,M1E,43.7678,-79.1866
3,Scarborough (Woburn),Ontario,M1G,43.7712,-79.2144
4,Scarborough (Cedarbrae),Ontario,M1H,43.7686,-79.2389
...,...,...,...,...,...
97,Weston,Ontario,M9N,43.7068,-79.5170
98,Etobicoke (Westmount),Ontario,M9P,43.6949,-79.5323
99,Etobicoke (Kingsview Village / St. Phillips / ...,Ontario,M9R,43.6898,-79.5582
100,Etobicoke (South Steeles / Silverstone / Humbe...,Ontario,M9V,43.7432,-79.5876


# Part 1

## Case 1 Solution

### Sets

In [ ]:
#restaurants
restaurants= list(dfOrdersA.index)

#customers
customers=list(dfOrdersA['customer'])

#number of trips
numtrips=len((customers+restaurants))

#restaurant-customer list
rescustlist=['Downtown Toronto (Rosedale)']+restaurants+customers

#steps
steps = list(range(1, len(rescustlist)))

#starting points
startarc = list(range(len(rescustlist)))

#ending points
endarc = steps.copy()

In [ ]:
regions = list(dfRegions["name"].unique())

distancedictionary = {}
for i in regions:
    for j in regions:
        if i == j:
            distancedictionary[(i,j)] = float(0)
        else: 
            distancedictionary[(i,j)] = float(dfDistance[(dfDistance["origin"] == i)&(dfDistance["destination"] == j)]["distance"])


### Parameters

### Variables

In [ ]:
#variables: if travelling from resteurant i to customer j at step t
xvar = pulp.LpVariable.dict("x", (startarc,endarc,steps), cat=pulp.LpBinary)


### Model Initialization

In [ ]:
#initialize model

flowModel = pulp.LpProblem(name="Case 1 Solution", sense=pulp.LpMinimize)

flowModel

/Users/shirley.zhang/opt/anaconda3/lib/python3.8/site-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


Case_1_Solution:
MINIMIZE
None
VARIABLES

### Constraints

In [ ]:
#Constraint: every location must be visited once
for j in endarc:
    flowModel+= pulp.lpSum( [xvar[i,j,t] for i in startarc for t in steps]) ==1

#Constraint: conservation of flow (go to node j in step t then start at node j in step t+1) 
for t in steps[:-1]:
    for j in endarc:
        flowModel += (pulp.lpSum([xvar[(i, j, t)]] for i in startarc) == pulp.lpSum([xvar[(j, k, t+1)]] for k in endarc))


# during the first step downtown Rosedale has an outflow of 1, others are zero
flowModel += pulp.lpSum([xvar[(0, j, 1)] for j in endarc]) == 1
for i in startarc:
    if i != 0:
        flowModel += pulp.lpSum([xvar[(i, j, 1)]  for j in endarc]) == 0

#Constraint: precedence must occur (must pick up food before dropping off at customer)
for t in steps:
    for j in endarc:
        if j > len(customers):
            if t == 1:
                flowModel += pulp.lpSum(xvar[i, j, t] for i in startarc) == 0 
            else:
                flowModel += pulp.lpSum(xvar[(i, j, t)] for i in startarc) <= pulp.lpSum(xvar[(i, j-len(customers), past_t)] for i in startarc for past_t in steps[:t-1])
  

### Objective Function

In [ ]:
objective=0
for i in startarc:
    for j in endarc:
        if i != j:
            objective += pulp.lpSum([distancedictionary[(rescustlist[i],rescustlist[j])] * xvar[(i, j, t)] for t in steps])

In [ ]:
# Solve model
flowModel+=objective
flowModel.solve()
print("Status:", pulp.LpStatus[flowModel.status])

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/shirley.zhang/opt/anaconda3/lib/python3.8/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/jj/335494hd5z5cblj975hs90400000gp/T/25cbda350c924cdd8abae1a9a130c1ed-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/jj/335494hd5z5cblj975hs90400000gp/T/25cbda350c924cdd8abae1a9a130c1ed-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 14 COLUMNS
At line 84 RHS
At line 94 BOUNDS
At line 107 ENDATA
Problem MODEL has 9 rows, 12 columns and 37 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 4.65356 - 0.00 seconds
Cgl0002I 5 variables fixed
Cgl0004I processed model has 0 rows, 0 columns (0 integer (0 of which binary)) and 0 elements
Cbc3007W No integer variables - nothing to do
Cuts at root node changed objective from 4.65356 to -1.79769e+308
Probing was tri

In [ ]:
# Print solution
print("Total distance:" + str(pulp.value(flowModel.objective))+'\n')

for t in steps:
    print(t)
    for i in startarc:
        for j in endarc:
            if xvar[i,j,t].varValue >= 1.0:
                if j <= len(customers):
                    print(f'Driving from {rescustlist[i]} To \n{rescustlist[j]} for pickup')
                else:
                    print(f'Driving from {rescustlist[i]} To \n{rescustlist[j]} for delivery')                         
    print()  

Total distance:4.65355901787117

1
Driving from Downtown Toronto (Rosedale) To 
Downtown Toronto (Underground city) for pickup

2
Driving from Downtown Toronto (Underground city) To 
Downtown Toronto (Central Bay Street) for delivery



## Case 2 Solution

### Sets

In [ ]:
#restaurants
resteurants= list(dfOrdersB.index)

#customers
customers=list(dfOrdersB['customer'])

#number of trips
numtrips=len((customers+resteurants))

#resteurant-customer list
rescustlist=['Downtown Toronto (Rosedale)']+resteurants+customers

#steps
steps = list(range(1, len(rescustlist)))

#starting points
startarc = list(range(len(rescustlist)))

#ending points
endarc = steps.copy()

### Parameters

### Variables

In [ ]:
#variables: if travelling from resteurant i to customer j at step t
xvar = pulp.LpVariable.dict("x", (startarc,endarc,steps), cat=pulp.LpBinary)


### Model Initialization

In [ ]:
#initialie model and objective sense

flowModel = pulp.LpProblem(name="Case 2 Solution", sense=pulp.LpMinimize)

flowModel

Case_2_Solution:
MINIMIZE
None
VARIABLES

### Constraints

In [ ]:
#Constraint: every location must be visited once
for j in endarc:
    flowModel+= pulp.lpSum( [xvar[i,j,t] for i in startarc for t in steps]) ==1

#Constraint: conservation of flow (go to node j in step t then start at node j in step t+1) 
for t in steps[:-1]:
    for j in endarc:
        flowModel += (pulp.lpSum([xvar[(i, j, t)]] for i in startarc) == pulp.lpSum([xvar[(j, k, t+1)]] for k in endarc))


# during the first step downtown rosedale has an outflow of 1, others are zero
flowModel += pulp.lpSum([xvar[(0, j, 1)] for j in endarc]) == 1
for i in startarc:
    if i != 0:
        flowModel += pulp.lpSum([xvar[(i, j, 1)]  for j in endarc]) == 0

#Constraint: precedence must occur (must pick up food before dropping off at customer)
for t in steps:
    for j in endarc:
        if j > len(customers):
            if t == 1:
                flowModel   += pulp.lpSum(xvar[(i, j, t)] for i in startarc) <= pulp.lpSum(xvar[(i, j-len(customers), past_t)] for i in startarc for past_t in steps[:t-1])
  

### Objective Function

In [ ]:
objective=0
for i in startarc:
    for j in endarc:
        if i != j:
            objective += pulp.lpSum([distancedictionary[(rescustlist[i],rescustlist[j])] * xvar[(i, j, t)] for t in steps])

In [ ]:
# Solve model
flowModel+=objective
flowModel.solve()
print("Status:", pulp.LpStatus[flowModel.status])

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/shirley.zhang/opt/anaconda3/lib/python3.8/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/jj/335494hd5z5cblj975hs90400000gp/T/840660c7d4df47c596fa53af3e10ca27-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/jj/335494hd5z5cblj975hs90400000gp/T/840660c7d4df47c596fa53af3e10ca27-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 121 COLUMNS
At line 6437 RHS
At line 6554 BOUNDS
At line 7655 ENDATA
Problem MODEL has 116 rows, 1100 columns and 3155 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 7.13608 - 0.00 seconds
Cgl0002I 105 variables fixed
Cgl0003I 75 fixed, 0 tightened bounds, 0 strengthened rows, 0 substitutions
Cgl0003I 2 fixed, 0 tightened bounds, 0 strengthened rows, 0 substitutions
Cgl0004I processed model has 95 rows, 867 columns (867 i

In [ ]:
# Print solution
print("Total distance:" + str(pulp.value(flowModel.objective))+'\n')

for t in steps:
    print(t)
    for i in startarc:
        for j in endarc:
            if xvar[i,j,t].varValue >= 1.0:
                if j <= len(customers):
                    print(f'Driving from {rescustlist[i]} To \n{rescustlist[j]} for pickup')
                else:
                    print(f'Driving from {rescustlist[i]} To \n{rescustlist[j]} for delivery')                         
    print()  

Total distance:28.51625850559942

1
Driving from Downtown Toronto (Rosedale) To 
Downtown Toronto (Central Bay Street) for pickup

2
Driving from Downtown Toronto (Central Bay Street) To 
Downtown Toronto (Central Bay Street) for pickup

3
Driving from Downtown Toronto (Central Bay Street) To 
Downtown Toronto (Richmond / Adelaide / King) for pickup

4
Driving from Downtown Toronto (Richmond / Adelaide / King) To 
Downtown Toronto (Richmond / Adelaide / King) for delivery

5
Driving from Downtown Toronto (Richmond / Adelaide / King) To 
Downtown Toronto (Underground city) for delivery

6
Driving from Downtown Toronto (Underground city) To 
Downtown Toronto (St. James Town / Cabbagetown) for delivery

7
Driving from Downtown Toronto (St. James Town / Cabbagetown) To 
Central Toronto (The Annex / North Midtown / Yorkville) for delivery

8
Driving from Central Toronto (The Annex / North Midtown / Yorkville) To 
York (Cedarvale) for pickup

9
Driving from York (Cedarvale) To 
Etobicoke Nor